# SK-67D104

Updating the dataset: P103130200000

* There is a strong worm at the long-wavelength end of the LiF 1B spectrum.  
* Used LiF 2A to rescale LiF 1B between 1180 and 1190 A.  
* Then scaled the entire spectrum (by 0.8) to match STIS.

COMMENT File updated 27 September 2022<br>
COMMENT 1080-1190 scaled to correct for worm<br>
COMMENT Spectrum scaled to match the STIS spectrum of this star.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify figure format.
quantity_support()  # to put units on the axes 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E12

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Read header keywords.

filename = 'p103130200000all4ttagfcal.fit'
f = fits.open(filename)
print ('Target:', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# LiF 1 is the guide channel.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Let's compare the two long-wavelength channels:

f, ax = plt.subplots()  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()

# At long wavelengths, LiF 1B suffers from an obvious worm.  We rescale it
# to match LiF 2A.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1175*u.AA, 1180*u.AA)
sub_lif1b = extract_region(lif1b, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes.

scale_lif1b = sub_lif2a.mean()/sub_lif1b.mean()
print ('Scale LiF 1b by', scale_lif1b)

In [ ]:
# Scale the smoothed spectrum.

lif1b_bsmooth *= scale_lif1b

In [ ]:
# Plot rescaled spectrum.

f, ax = plt.subplots()  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.legend()

# LiF 1B is now OK at longest wavelengths.

In [ ]:
# Let's compare LiF 2A with a STIS spectrum of the same star.

filename = 'o4yn02010_x1d.fits'
print ('Target:', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.plot(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A')
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,10])

# The FUSE spectrum is a bit brigher than the STIS spectrum.  We will rescale to match.

In [ ]:
# Let's compute the scale factor between the STIS and LiF 2A spectrum.
# Select a spectral region.

region = SpectralRegion(1150*u.AA, 1160*u.AA)
sub_stis = extract_region(stis, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes.

scale_to_match_stis = norm * sub_stis.mean()/sub_lif2a.mean()
print ('Scale by', scale_to_match_stis)

# We will apply this scale factor to the entire spectrum at the very end.

In [ ]:
# Plot the STIS and the rescaled FUSE spectrum.

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.plot(lif2a_bsmooth.spectral_axis, scale_to_match_stis * lif2a_bsmooth.flux, label='LiF 2A')
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,10])

# Note the apparent P Cygni profile near 1170 A, which in the FUSE spectrum but not in STIS. 
# Otherwise, the match is good.

In [ ]:
# Apply this correction to the NVO file.

filename = 'p103130200000nvo4ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale LiF 1B.
g = np.where(data['wave'] > 1180)
data['flux'][g] *= float(scale_lif1b)
data['error'][g] *= float(scale_lif1b)

# Now scale data and error arrays to match STIS.
data['flux'] *= float(scale_to_match_stis)
data['error'] *= float(scale_to_match_stis)

hdr['comment'] = ''
hdr['comment'] = 'File updated 27 September 2022'
hdr['comment'] = '1080-1190 scaled to correct for worm'
hdr['comment'] = 'Spectrum scaled to match the STIS spectrum of this star.'

f.writeto('level0_p103130200000nvo4ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'p103130200000nvo4ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 

filename = 'level0_p103130200000nvo4ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])